In [1]:
import sys
from time import time
from scipy import stats, sparse
from scipy.sparse.linalg import svds, eigs
from scipy.special import expit
import numpy as np
import os
import pandas as pd
import collections

In [2]:
import OnClass

In [3]:
#from OnClass.utils import *
from sentence_transformers import SentenceTransformer
from scipy import spatial

2023-01-30 22:30:43.989911: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-30 22:30:44.107081: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-30 22:30:44.700221: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:
2023-01-30 22:30:44.700295: W tensorflow/compiler/xla/stream_executor/platf

In [7]:
net = collections.defaultdict(dict)
rev_net = collections.defaultdict(dict)

fin = open('../ontology/cl.ontology')

for line in fin:
	s,p = line.lower().strip().split('\t') #p is s' parent
	net[s][p] = 1
	rev_net[p][s] = 1
fin.close()

In [12]:
model = SentenceTransformer('all-mpnet-base-v2')

id2sent = {}
fin = open('../ontology/cl.obo')
for line in fin:
	if line.lower().startswith('def: '):
		sent = line.lower().strip().split('"')[1]
		id2sent[id] = sent
	if line.lower().startswith('name: '):
		name = line.lower().strip().split('name: ')[1]
		id2sent[id] = name
	if line.lower().startswith('id: '):
		id = line.lower().strip().split('id: ')[1]

sentences = list(id2sent.values())
sentence_embeddings = model.encode(sentences)

sent2vec = {}
for sentence, embedding in zip(sentences, sentence_embeddings):
    sent2vec[sentence] = embedding


Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/571 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/116 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/363 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/13.1k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/349 [00:00<?, ?B/s]

In [19]:
fout = open('cl.ontology.nlp.emb','w')
for id in id2sent:
	fout.write(id.upper())
	vec = sent2vec[id2sent[id]]
	for v in vec:
		fout.write('\t'+str(v))
	fout.write('\n')
fout.close()